# Build a Web Research Agent

In the first exercise, you learned the basics of the
Tavily API. Now, let's take it a step further: we'll
combine that knowledge with LLMs to unlock
real value. In this tutorial, you'll learn how to build a web research agent that can search, extract, crawl, and reason over live web data.

You already explored the various parameter
configurations available for each Tavily API endpoint.
With the official [Tavily-LangChain integration](https://www.tavily.com/integrations/langchain) integration, our agent can
automatically set these parameters—like the
`time_range` for search or specific crawl
`instructions`—based on the context and requirements of
each task. This dynamic, LLM-powered configuration is
powerful in agentic systems.

By the end of this lesson, you'll know how to:
- Seamlessly connect foundation models to the web for up-to-date research
- Build a react-style web agent
- Dynamically configure search, extract, and crawl parameters the Tavily-LangChain integration.



---

## Getting Started

Follow these steps to set up:

1. **Sign up** for Tavily at https://www.tavily.com/

Install dependencies in the cell below.

In [1]:
%pip install -U tavily-python langchain-openai langchain langchain-tavily langgraph --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 8.3 MB/s eta 0:00:00


### Setting Up Your Tavily API Client

The code below will instantiate the Tavily client with your API key.

In [2]:
import os
import getpass
from tavily import TavilyClient

# Prompt the user to securely input the API key if not already set in the environment

os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI API Key:\n")

# Initialize the Tavily API client using the loaded or provided API key
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

TAVILY_API_KEY:
··········
OPENAI API Key:
··········


Let's define the following modular tools with the Tavily-LangChain integration:
1. **Search** the web for relevant information

2. **Extract** content from specific web pages

3. **Crawl** entire websites

In [3]:
# Define the set of web tools our agent will use to interact with the Tavily API.
from langchain_tavily import TavilySearch
from langchain_tavily import TavilyExtract
from langchain_tavily import TavilyCrawl

# Define the LangChain search tool
search = TavilySearch(max_results=10, topic="general")

# Define the LangChain extract tool
extract = TavilyExtract(extract_depth="advanced")

# Define the LangChain crawl tool
crawl = TavilyCrawl()

Now let's set up several OpenAI foundation models to power our agent, such as o3-mini and the gpt-4.1 model. If you prefer a different LLM provider, you can easily plug in any LangChain Chat Model.

In [4]:
# Instantiate the OpenAI foundation models
from langchain_openai import ChatOpenAI

# o3-mini-2025-01-31
o3_mini = ChatOpenAI(model="o3-mini-2025-01-31", api_key=os.getenv("OPENAI_API_KEY"))

# gpt-4.1
gpt_4_1 = ChatOpenAI(model="gpt-4.1", api_key=os.getenv("OPENAI_API_KEY"))

## Web Agent

Next, we'll build a Web Agent powered by Tavily, which consists of three main components: the language model, a set of web tools, and a system prompt. The language model (such as o3-mini or gpt-4.1) serves as the agent's "brain," while the web tools (Search, Extract, and Crawl) allow the agent to interact with and gather information from the internet. The system prompt guides the agent's behavior, explaining how and when to use each tool to accomplish its research goals.

This agent leverages a pre-built LangGraph reAct implementation. The reAct framework enables the agent to reason about which actions to take, use the web tools in sequence, and iterate as needed until it completes its research task. The system prompt is especially important—it instructs the agent on best practices for using the tools together, ensuring that the agent's responses are thorough, accurate, and well-sourced.

You are encouraged to experiment with the system prompt or try different language models (like swapping between gpt-4.1 and o3-mini) to change the agent's style, personality, or optimize its performance for specific use cases.


In [5]:
import datetime
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

today = datetime.datetime.today().strftime("%A, %B %d, %Y")

# Create the web agent
web_agent = create_react_agent(
    model=gpt_4_1,
    tools=[search, extract, crawl],
    prompt=ChatPromptTemplate.from_messages(
        [
            (
                "system",
                f"""
        You are a research agent equipped with advanced web tools: Tavily Web Search, Web Crawl, and Web Extract. Your mission is to conduct comprehensive, accurate, and up-to-date research, grounding your findings in credible web sources.

        **Today's Date:** {today}

        **Available Tools:**

        1. **Tavily Web Search**

        * **Purpose:** Retrieve relevant web pages based on a query.
        * **Usage:** Provide a search query to receive semantically ranked results, each containing the title, URL, and a content snippet.
        * **Best Practices:**

            * Use specific queries to narrow down results.
            * Optimize searches using parameters such as `search_depth`, `time_range`, `include_domains`, and `include_raw_content`.
            * Break down complex queries into specific, focused sub-queries.

        2. **Tavily Web Crawl**

        * **Purpose:** Explore a website's structure and gather content from linked pages for deep research and information discovery from a single source.
        * **Usage:** Input a base URL to crawl, specifying parameters such as `max_depth`, `max_breadth`, and `extract_depth`.
        * **Best Practices:**

            * Begin with shallow crawls and progressively increase depth.
            * Utilize `select_paths` or `exclude_paths` to focus the crawl.
            * Set `extract_depth` to "advanced" for comprehensive extraction.

        3. **Tavily Web Extract**

        * **Purpose:** Extract the full content from specific web pages.
        * **Usage:** Provide URLs to retrieve detailed content.
        * **Best Practices:**

            * Set `extract_depth` to "advanced" for detailed content, including tables and embedded media.
            * Enable `include_images` if image data is necessary.

        **Guidelines for Conducting Research:**

        * **Citations:** Always support findings with source URLs, clearly provided as in-text citations.
        * **Accuracy:** Rely solely on data obtained via provided tools—never fabricate information.
        * **Methodology:** Follow a structured approach:

        * **Thought:** Consider necessary information and next steps.
        * **Action:** Select and execute appropriate tools.
        * **Observation:** Analyze obtained results.
        * Repeat Thought/Action/Observation cycles as needed.
        * **Final Answer:** Synthesize and present findings with citations in markdown format.

        **Example Workflows:**

        **Workflow 1: Search Only**

        **Question:** What are recent news headlines about artificial intelligence?

        * **Thought:** I need quick, recent articles about AI.
        * **Action:** Use Tavily Web Search with the query "recent artificial intelligence news" and set `time_range` to "week".
        * **Observation:** Retrieved 10 relevant articles from reputable news sources.
        * **Final Answer:** Summarize key headlines with citations.

        **Workflow 2: Search and Extract**

        **Question:** Provide detailed insights into recent advancements in quantum computing.

        * **Thought:** I should find recent detailed articles first.
        * **Action:** Use Tavily Web Search with the query "recent advancements in quantum computing" and set `time_range` to "month".
        * **Observation:** Retrieved 10 relevant results.
        * **Thought:** I should extract content from the most comprehensive article.
        * **Action:** Use Tavily Web Extract on the most relevant URL from search results.
        * **Observation:** Extracted detailed information about quantum computing advancements.
        * **Final Answer:** Provide detailed insights summarized from extracted content with citations.

        **Workflow 3: Search and Crawl**

        **Question:** What are the latest advancements in renewable energy technologies?

        * **Thought:** I need recent articles about advancements in renewable energy.
        * **Action:** Use Tavily Web Search with the query "latest advancements in renewable energy technologies" and set `time_range` to "month".
        * **Observation:** Retrieved 10 articles discussing recent developments in solar panels, wind turbines, and energy storage.
        * **Thought:** To gain deeper insights, I'll crawl a relevant industry-leading renewable energy site.
        * **Action:** Use Tavily Web Crawl on the URL of a leading renewable energy industry website, setting `max_depth` to 2.
        * **Observation:** Gathered extensive content from multiple articles linked on the site, highlighting new technologies and innovations.
        * **Final Answer:** Provide a synthesized summary of findings with citations.

        ---

        You will now receive a research question from the user:

        """,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    ),
    name="web_agent",
)

        **Today's Date:** {today}

        **Available Tools:**

        1. **Tavily Web Search**

        * **Purpose:** Retrieve relevant web pages based on a query.
        * **Usage:** Provide a search query to receive semantically ranked results, each containing the title, URL, and a content snippet.
        * **Best Practices:**

            * Use specific queries to narrow down results.
            * Optimize searches using parameters such as `search_depth`, `time_range`, `include_domains`, and `include_raw_content`.
            * Break down complex queries into specific, focused sub-queries.

### Test Your Tavily Web Agent

Now we'll run the agent and see how it uses the different web tools.

In [6]:
from langchain.schema import HumanMessage

# Test the web agent
inputs = {
    "messages": [
        HumanMessage(
            content="find all the iphone models currently available on apple.com and their prices"
        )
    ]
}

# Stream the web agent's response
for s in web_agent.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

find all the iphone models currently available on apple.com and their prices
================================== Ai Message ==================================
Name: web_agent
Tool Calls:
  tavily_search (call_qRNWZ6VrIl6hRWAeQyjDD1uk)
 Call ID: call_qRNWZ6VrIl6hRWAeQyjDD1uk
  Args:
    query: current iPhone models and prices site:apple.com
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "current iPhone models and prices", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://news.macgasm.net/iphone-news/iphone-17-expectations/", "title": "What to Expect From Apple's iPhone 17 Lineup This September", "content": "As for pricing, don’t hold your breath for anything “affordable.” The iPhone 17 will probably stay around the $799 mark, while the new iPhone 17 Air is expected to cost somewh

Examine the agent's intermediate steps printed above, including how it chooses and configures different tool parameters. Then, display the agent's final answer in markdown format.

In [7]:
from IPython.display import Markdown

Markdown(message.content)

Based on the most current official information from Apple’s website, here are all the iPhone models currently available for purchase directly from Apple in the United States, along with their starting prices:

### Currently Available iPhone Models and Their Prices (as of September 2025):

1. **iPhone 17 Pro**
   - Starting at $1,099  
2. **iPhone 17**
   - Starting at $799  
3. **iPhone Air** (the new mid-tier model, thinner than the Plus, new for 2025)
   - Starting at $999  
4. **iPhone 16**
   - Starting at $699  
5. **iPhone 16e** (the “entry” or value model, successor to the SE line)
   - Starting at $599  
6. **iPhone 15** (previous generation, still sold by Apple)
   - Starting at $699

> - All these base prices are for the entry-level storage options, and actual prices may vary depending on trade-in, financing options and activation requirements for connectivity discounts.

**Source:**  
- [Apple Official iPhone Shop Page](https://www.apple.com/us/shop/goto/buy_iphone)
- [Apple iPhone Product Page](https://www.apple.com/iphone/)

You can see more details, options, and offers directly on Apple's official store: [apple.com/us/shop/goto/buy_iphone](https://www.apple.com/us/shop/goto/buy_iphone).

---

**Summary Table:**

| Model          | Starting Price |
|----------------|:-------------:|
| iPhone 17 Pro  |   $1,099      |
| iPhone 17      |   $799        |
| iPhone Air     |   $999        |
| iPhone 16      |   $699        |
| iPhone 16e     |   $599        |
| iPhone 15      |   $699        |

For complete pricing, storage configurations, and current deals, always refer to the [official Apple store page](https://www.apple.com/us/shop/goto/buy_iphone).

Let's try a different example.

In [ ]:
# Test the web agent
inputs = {
    "messages": [
        HumanMessage(
            content="return 5 job postings for a software engineer in the bay area on linkedin"
        )
    ]
}

# Stream the web agent's response
for s in web_agent.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

return 5 job postings for a software engineer in the bay area on linkedin
================================== Ai Message ==================================
Name: web_agent
Tool Calls:
  tavily_search (call_MUo8mp9sRD8gq6ia9IK1URHJ)
 Call ID: call_MUo8mp9sRD8gq6ia9IK1URHJ
  Args:
    query: software engineer jobs Bay Area site:linkedin.com/jobs
    search_depth: advanced
    time_range: week
================================= Tool Message =================================
Name: tavily_search

{"query": "software engineer jobs Bay Area site:linkedin.com/jobs", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.linkedin.com/jobs/view/software-engineer-developer-infra-at-tecton-4269808648", "title": "Software Engineer, Developer Infra - Tecton", "content": "Get notified about new Software Engineer jobs in San Francisco Bay Area. ... Java Specialist jobs · Dotnet Develope

Print the agent's final response as markdown.

In [ ]:
# Print the agent's final response as markdown
Markdown(message.content)

Here are 5 recent software engineer job postings in the Bay Area from LinkedIn:

1. Software Engineer, Developer Infra at Tecton  
   Location: San Francisco Bay Area  
   [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-developer-infra-at-tecton-4269808648)

2. Software Engineer I (Front End/Full Stack), Entry Level at Jobright.ai  
   Location: San Francisco Bay Area  
   [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-i-front-end-full-stack-entry-level-at-jobright-ai-4269775633)

3. Software Engineer - New Grad (Internship) at Jobright.ai  
   Location: San Francisco Bay Area  
   [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-new-grad-internship-at-jobright-ai-4268514143)

4. Software Engineer at Understanding Recruitment  
   Location: San Francisco Bay Area  
   [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-at-understanding-recruitment-4267476299)

5. Software Engineer, Product (Entry Level) at Jobright.ai  
   Location: San Francisco Bay Area  
   [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-product-entry-level-at-jobright-ai-4270230839)

(Sources: LinkedIn job postings as of the past week)

Notice how the agent cleverly combines Tavily’s tools—search, crawl, and extract—to complete the task end-to-end.